<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import timeit
import math
import random
from scipy.optimize import fsolve
from mpmath import *
from datetime import datetime, timedelta
from scipy.optimize import curve_fit
#import xarray

In [2]:
def Lempel_Ziv2(usr, lambdas=False, e=100, **kwargs):
    """Estimate the entropy rate of the symbols encoded in `seq`, a list of
    strings.

    Kontoyiannis, I., Algoet, P. H., Suhov, Y. M., & Wyner, A. J. (1998).
    Nonparametric entropy estimation for stationary processes and random
    fields, with applications to English text. IEEE Transactions on Information
    Theory, 44(3), 1319-1327.

    Bagrow, James P., Xipei Liu, and Lewis Mitchell. "Information flow reveals
    prediction limits in online social activity." Nature human behaviour 3.2
    (2019): 122-128.
    """
    
    if 'locs' in kwargs:
        seq = wf3[(wf3['userid']==usr) & wf3['placeid'].isin(kwargs['locs'])]['placeid'].to_list()
        N = len(seq)
    else:
        seq = wf3[wf3['userid']==usr]['placeid'].to_list()
        N = len(seq)
    wb = 0

    if N < e:
        return [np.nan]*5
    else:
        L = []
        L.append(0)
        for i, w in enumerate(seq):
            prevSeq = " %s " % " ".join(seq[0:i])
            seen = (" %s " % " ".join(seq[i:i+1])) in prevSeq
            c = i
            while seen and c < N:
                c += 1
                seen = (" %s " % " ".join(seq[i:c+1])) in prevSeq
            
            l = c - i
            L.append(l)
        wb = len([x for x in L if x != 0])
        if lambdas:
            return L
        return [(1.0 * wb / sum(L)) * np.log2(N-1),sum(L),N,wb,L]
                #,L,np.sum(L),prevSeq]


In [3]:
def CrossEntropy_4(ego,alters, lambdas=False, e=100, **kwargs):
    
    """Estimate the entropy rate of the symbols encoded in `seq`, a list of
    strings.

    Kontoyiannis, I., Algoet, P. H., Suhov, Y. M., & Wyner, A. J. (1998).
    Nonparametric entropy estimation for stationary processes and random
    fields, with applications to English text. IEEE Transactions on Information
    Theory, 44(3), 1319-1327.

    Bagrow, James P., Xipei Liu, and Lewis Mitchell. "Information flow reveals
    prediction limits in online social activity." Nature human behaviour 3.2
    (2019): 122-128.
    """
#============================================================================================================================

    '''
    returns[array of cross entropies, array of weights wb (returns len(alters) + 1 elements if with ego == True),Array of Lambda_i]
    '''
    '''Dictionaries of Alters and their lengths are stored in an array in the order the alters were called
    kwargs:
        with_ego: bool, True implies we include the ego in the cummulative cross entropy
        temporal_control: bool, True means we shuffle the time stamps of the alter locations
    '''
    
    '''Lambda_i is a list of the cross-parsed match lengths of the ego based on each alter i
    wb is a list of number of matches of substrings of A in B
    cross_ent is the list of (cummulative) cross entropies of the alters'''
    
    TempCont = False
    if 'temporal_control' in kwargs:
        TempCont = kwargs['temporal_control']
    '''Gets Coordinates of alters. Makes array of x-locs and y-locs
    key is an array the size of the list of locations with all elements 'B', signifying each element as the alter's
    N_alters is a list of the number of coordinates in the alter's string
    Time_alters are the timestamps of the location visits
    '''
    
    
    #key denotes whether the element in the sequence is the ego's datum or alter's datum
    
    seq_ego = wf3[wf3['userid'] == ego]['placeid'].to_list()
    key_ego = ['A']*len(seq_ego)
    time_ego = wf3[wf3['userid'] == ego]['datetime'].to_list()
    N_ego = len(seq_ego)
    
    if N_ego < e:
        return float('nan')
    
    # Reading in the sequences of the alter(s)
    
    if type(alters) is list:
        seq_alters = []
        key_alters = []
        time_alters = []
        N_alters = []
        k = 0
        for usr in alters:
            #print(usr)
            seq_alters.append(wf3[wf3['userid'] == usr]['placeid'].to_list())
            key_alters.append(['B']*len(seq_alters[k]))
            time_alters.append(wf3[wf3['userid'] == usr]['datetime'].to_list())
            N_alters.append(len(seq_alters[k]))
            if TempCont:
                '''If we want a temporally controlled entropy, we shuffle the times and sort the 
                locations with respect to the shuffled time list'''
                random.shuffle(time_alters[k])
                seq_alters[k] = [x for _, x in sorted(zip(time_alters[k],seq_alters[k]))]
                seq_alters[k] = [x for _, x in sorted(zip(time_alters[k],seq_alters[k]))]
            k+=1
    else:
        k=0
        seq_alters = [wf3[wf3['userid'] == alters]['placeid'].to_list()]
        key_alters = [['B']*len(seq_alters[k])]
        time_alters = [wf3[wf3['userid'] == alters]['datetime'].to_list()]
        N_alters = [len(seq_alters[0])]
        if TempCont:
            random.shuffle(time_alters)
            seq_alters = [x for _, x in sorted(zip(time_alters,seq_alters))]
    
    

#============================================================================================================
#============================================================================================================
 

    L_i = []
    Lambda_max = []
    wb = []
    cumcross_ent = []
    cross_ents = []
    mut_unique =[]
    num_mut_unique = []
    cum_mut_unique = []
    cumnum_mut_unique = []
    N_Ego_Seen = []
    
    #L_i = {L_i}
    #Lambda_max = max({Lambda_i} for all i for all  alters)
    #wb = list of weights of pairs
    #cross ents = list of cross entropies
    #mut_unqiue = list of lists of mutual locations shared between an ego and alter
    #num_mut_unique = {len(mut_unique[i])} for all i
    #cum_mut_unique = list of cummulative shared mutual locations betwen the ego and all alters
    #cumnum_mut_unique = length of ^
    #N_Ego_Seen = the number of elements seen by the ego that have been previously seen by all alters, i.e. len(Lambda_max)
    #where Lambda_max_i != 0

#============================================================================================================


    
    
    k = -1
    ego_index = 0
    if 'with_ego' in kwargs:
        with_ego = kwargs['with_ego']
        if kwargs['with_ego']:
            dummy = Lempel_Ziv(ego)
            N_alters.insert(0,dummy[2])
            wb.append(dummy[2])
            L_i.append(dummy[4])
            ego_index = 1
    else:
        with_ego = False 

#------------------------------------------------------------------------------------------------
 
    for ALTER in seq_alters:
        i = 0
        i_ego = 0
        i_alter = 0
        k+=1
        dict_ego = []
        dict_alter = []
        mut_unique.append([])
        wb.append(0)
        num_mut_unique.append(0)
        
#Sort sequences with respect to time, preserving the order of the individual ego/alter sequences through the key
        seq = seq_alters[k] + seq_ego
        key = key_alters[k] + key_ego
        times = time_alters[k] + time_ego
        key = [x for _, x in sorted(zip(times,key))]
        seq = [x for _, x in sorted(zip(times,seq))]
        times = sorted(times)
        N_alters[k+ego_index] = N_alters[k+ego_index] - key[::-1].index('A')
        seq = seq[:(len(key)-key[::-1].index('A'))]
        key = key[:(len(key)-key[::-1].index('A'))]
        
        #If after intializing the sequences the alter's string length is not long enough, go to the next alter
        if N_alters[k + ego_index] < 200:
            N_alters[k + ego_index] = 0
            wb[-1] = 0
            L_i.append([0]*N_ego)
            cross_ents.append(float('nan'))
        else:
            i_ego = 0
            i_alter = 0
            prevSeq = 0
            L_i.append([])
            for i, w in enumerate(seq):
                if key[i] == 'B':
                    i_alter += 1
                else:
                    prevSeq = " %s " % " ".join(seq_alters[k][0:i_alter])
                    seen = (" %s " % " ".join(seq_ego[i_ego:i_ego+1])) in prevSeq
                    c = i_ego
                    while seen and c < N_ego:   
                        c += 1
                        seen = (" %s " % " ".join(seq_ego[i_ego:c+1])) in prevSeq
                        if seq_ego[c - 1] not in mut_unique[-1]:
                            num_mut_unique[-1] += 1
                            mut_unique[-1].append(seq_ego[c - 1])
                
                    l = c - i_ego
                    i_ego += 1
                    L_i[-1].append(l)
                
            wb[-1] = len([x for x in L_i[-1] if x != 0])
            
            
            if wb[-1] < e:
                N_alters[k + ego_index] = 0
                L_i[-1] = [0]*len(L_i[-1])
                cross_ents.append(float('nan'))
            else:
                cross_ents.append(wb[-1]*np.log2(N_alters[k+ego_index])/sum(L_i[-1]))

#--------------------------------------------------------------------------------------------------------------
                
                
        if with_ego == True:
            N_ego = len([x for x in L_i[-1] if x != 0])
            N_AB_one = (wb[0]*N_ego + wb[-1]*N_alters[k + ego_index])/(wb[0]+wb[-1])
            Lambda_max_one = np.sum(np.nanmax([L_i[0],L_i[-1]],axis=0))
            if (N_alters[k + ego_index] < e) or (wb[-1] < e):
                cross_ents.append(float('nan'))
            else:
                cross_ents.append(N_ego*np.log2(N_AB_one)/Lambda_max_one)
        #Finding the Cumulative Unique Locations
        if cum_mut_unique == []:
            cum_mut_unique.append(mut_unique[-1])
            cumnum_mut_unique.append(num_mut_unique[-1])
        else:
            cum_mut_unique.append(list(set(mut_unique[-1] + cum_mut_unique[-1])))
            cumnum_mut_unique.append(len([x for x in mut_unique[-1] if x not in cum_mut_unique[-2]]))
        
        #Calculating CCE
        N_AB = np.sum(np.multiply(wb,N_alters[:len(wb)]))/np.sum(wb)
        Lambda_max = np.nanmax(L_i,axis=0)
        N_Ego_Seen.append(len([x for x in Lambda_max if x != 0]))
        cumcross_ent.append(N_Ego_Seen[-1]*np.log2(N_AB)/np.sum(Lambda_max))
        
        
        if mut_unique[-1] == []:
            mut_unique[-1] = [float('nan')]
            if cum_mut_unique[-1] == []:
                cum_mut_unique[-1] = [float('nan')]

                
#============================================================================================================
#============================================================================================================
 
    if lambdas:
        return L
    return [cumcross_ent,cross_ents,wb[ego_index:],N_alters[ego_index:],num_mut_unique,np.cumsum(cumnum_mut_unique),mut_unique,cum_mut_unique,N_Ego_Seen,list(Lambda_max)]


In [4]:
'''Fano Inequality'''
def Fano(Pi_max, N, S):
    return np.log2(N-1)-S+Pi_max*np.log2((1/Pi_max - 1)*(1/(N-1))) - np.log2(1-Pi_max)